<a href="https://colab.research.google.com/github/toraaglobal/fashion-mnist/blob/master/02_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### MODEL

In [1]:
## Mount the gdrive
import os
from google.colab import drive
drive.mount('/content/drive')


## change directry to the gdrive

os.chdir('./drive/My Drive/Colab Notebooks/lab3')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
## import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time


## model
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier  ##min_samples_split, max_leaf_nodes, max_depth and min_samples_leaf.
from sklearn.ensemble import GradientBoostingClassifier  ## The learning_rate is a hyper-parameter in the range (0.0, 1.0] 
                                                        ##that controls overfitting 

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier  ## clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
'''
SGDClassifier(alpha=0.0001, average=False, class_weight=None,
           early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
           l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
           n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
           random_state=None, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)
'''



#import tensorflow as tf


from mlp import NeuralNetMLP
from mlp import MLPGradientCheck
import mlp

from sklearn.pipeline import Pipeline
import os
import struct

In [3]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

from kkeras import baseline_model

Using TensorFlow backend.


In [4]:
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, '%s-labels-idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images-idx3-ubyte' % kind)
        
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII", imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
 
    return images, labels

## load traning set
X_train, y_train = load_mnist('./data/', kind='train')
print('Rows: %d, columns: %d' % (X_train.shape[0], X_train.shape[1]))

## load test set
X_test, y_test = load_mnist('./data/', kind='t10k')
print('Rows: %d, columns: %d' % (X_test.shape[0], X_test.shape[1]))

Rows: 60000, columns: 784
Rows: 10000, columns: 784


In [0]:
## create a model container and emty list to store training and prediction accuracy
model = {}
model_type = []
training_score = []
test_score = []
training_time = []
prediction_time = []

In [0]:

# SCRUB
# FLATTEN 28 x 28 IMAGE TO 784 VECTOR
num_pixels = X_train.shape[1]
#X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
#X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')



# SCRUB
# NORMALIZE INPUTS FROM RGB COLOR TO 0-1
X_train = X_train / 255
X_test = X_test / 255


# SCRUB
# THE OLD ONE HOT ENCODE - CONVERT "CATEGORICAL" CLASSIFICATION TO ENCODE
# A "BINARIZATION" OF THE CATEGORIES
y_train_k = np_utils.to_categorical(y_train)
y_test_k = np_utils.to_categorical(y_test)
num_classes = 10

In [7]:

## initialize model and add model to container
model['GaussianNB'] = GaussianNB()
model['RandomForest'] = RandomForestClassifier()
model['GradientBoostingClassifier'] = GradientBoostingClassifier()
model['nnMLP'] = NeuralNetMLP(n_output=10, 
                  n_features=X_train.shape[1], 
                  n_hidden=50, 
                  l2=0.1, 
                  l1=0.0, 
                  epochs=200, 
                  eta=0.001,
                  alpha=0.001,
                  decrease_const=0.00001,
                  minibatches=50, 
                  shuffle=True,
                  random_state=1)


#model['MLPGradientCheck'] = MLPGradientCheck(n_output=10, 
#                            n_features=X_train.shape[1], 
#                            n_hidden=10, 
#                            l2=0.0, 
#                            l1=0.0, 
#                            epochs=100, 
#                            eta=0.001,
#                            alpha=0.0,
#                            decrease_const=0.0,
#                            minibatches=1, 
#                            shuffle=False,
#                            random_state=1)

model['keras_adam'] = baseline_model(num_pixels,num_classes, optimizer='adam',metrics=['accuracy'])

model['keras_sgd'] = baseline_model(num_pixels,num_classes, optimizer='sgd',metrics=['accuracy'])

model['keras_Adagrad'] = baseline_model(num_pixels,num_classes, optimizer='Adagrad',metrics=['accuracy'])

model['keras_Adadelta'] =baseline_model(num_pixels,num_classes, optimizer='Adadelta',metrics=['accuracy'])




Instructions for updating:
Colocations handled automatically by placer.


In [11]:
from sklearn.metrics import accuracy_score
process_start = time.time()
for mod in model:
    print("="* 50)
    print(mod)
    print("="* 50)

    ## start the model
    
    ## check non keras model
    if mod not in ('keras_adam','keras_sgd','keras_Adagrad','keras_Adadelta'):
        train_start = time.time()
        model[mod].fit(X_train,y_train)
        train_end = time.time()
    else:
        train_start = time.time()
        model[mod].fit(X_train,y_train_k, epochs=200)
        train_end = time.time()
        
    
    print("{} training time {}".format(mod,train_end-train_start))
    
    ## train score
    if mod not in ('keras_adam','keras_sgd','keras_Adagrad','keras_Adadelta'):
        train_prediction = model[mod].predict(X_train)
        train_score =  accuracy_score(train_prediction, y_train)
    else:
        #train_prediction = model[mod].predict(X_train)
        train_score =  model[mod].evaluate(X_train, y_train_k)
        
    print("{} training accuracy {}".format(mod, train_score))
    
    ## test prediction
    if mod not in ('keras_adam','keras_sgd','keras_Adagrad','keras_Adadelta'):
        predict_start = time.time()
        prediction = model[mod].predict(X_test)
        predict_end = time.time()
        score = accuracy_score(prediction, y_test)
    else:
        predict_start = time.time()
        #prediction = model[mod].predict(X_test)
        score = model[mod].evaluate(X_test, y_test_k)
        predict_end = time.time()
        
     
    ## score test
    print("{} time for testing {}".format(mod,predict_end-predict_start))
    
    print("{} test accuracy {}".format(mod, score))
    
    ## append all results
    model_type.append(mod)
    training_score.append(train_score)
    test_score.append(score)
    training_time.append(train_end - train_start)
    prediction_time.append(predict_end -  predict_start)
    print("Done with {}".format(mod))
    

process_end = time.time()      
## create a dataframe
result = {'Model': model_type, 'Training Accuracy': training_score, 'Training Time': training_time, "Test Accuracy": test_score,
          "Prediction Time": prediction_time}
df = pd.DataFrame(result)

print("Total Process Time: {}".format(process_end -  process_start))
df


GaussianNB
GaussianNB training time 0.7682697772979736
GaussianNB training accuracy 0.5877833333333333
GaussianNB time for testing 0.6917402744293213
GaussianNB test accuracy 0.5856
Done with GaussianNB
RandomForest
RandomForest training time 11.132927179336548
RandomForest training accuracy 0.9956
RandomForest time for testing 0.0856635570526123
RandomForest test accuracy 0.8537
Done with RandomForest
GradientBoostingClassifier
GradientBoostingClassifier training time 5901.939537763596
GradientBoostingClassifier training accuracy 0.9029
GradientBoostingClassifier time for testing 0.39362597465515137
GradientBoostingClassifier test accuracy 0.8668
Done with GradientBoostingClassifier
nnMLP
nnMLP training time 223.91688418388367
nnMLP training accuracy 0.9200666666666667
nnMLP time for testing 0.09986495971679688
nnMLP test accuracy 0.8809
Done with nnMLP
keras_adam
Epoch 1/200
60000/60000 [==============================] - 6s 94us/step - loss: 0.0263 - acc: 0.9920
Epoch 2/200
60000/600

,Model,Training Accuracy,Training Time,Test Accuracy,Prediction Time
0,GaussianNB,0.587783,1.141622,0.5856,0.657928
1,RandomForest,0.99515,10.502508,0.8546,0.071329
2,GradientBoostingClassifier,0.9029,5638.623037,0.8669,0.621780
3,nnMLP,0.90255,265.021065,0.8688,0.101179
4,GaussianNB,0.587783,0.768270,0.5856,0.691740
5,RandomForest,0.9956,11.132927,0.8537,0.085664
6,GradientBoostingClassifier,0.9029,5901.939538,0.8668,0.393626
7,nnMLP,0.920067,223.916884,0.8809,0.099865
8,keras_adam,"[0.011163142539572974, 0.9968333333333333]",1177.883724,"[1.15368981669005, 0.8943]",0.341279
9,keras_sgd,"[0.038993409626434244, 0.9930833333333333]",946.155392,"[0.3552287856295705, 0.8973]",0.364729


In [12]:
result = {'Model': model_type, 'Training Accuracy': training_score, 'Training Time': training_time, "Test Accuracy": test_score,
          "Prediction Time": prediction_time}
df = pd.DataFrame(result)

#print("Total Process Time: {}".format(process_end -  process_start))
df


,Model,Training Accuracy,Training Time,Test Accuracy,Prediction Time
0,GaussianNB,0.587783,1.141622,0.5856,0.657928
1,RandomForest,0.99515,10.502508,0.8546,0.071329
2,GradientBoostingClassifier,0.9029,5638.623037,0.8669,0.621780
3,nnMLP,0.90255,265.021065,0.8688,0.101179
4,GaussianNB,0.587783,0.768270,0.5856,0.691740
5,RandomForest,0.9956,11.132927,0.8537,0.085664
6,GradientBoostingClassifier,0.9029,5901.939538,0.8668,0.393626
7,nnMLP,0.920067,223.916884,0.8809,0.099865
8,keras_adam,"[0.011163142539572974, 0.9968333333333333]",1177.883724,"[1.15368981669005, 0.8943]",0.341279
9,keras_sgd,"[0.038993409626434244, 0.9930833333333333]",946.155392,"[0.3552287856295705, 0.8973]",0.364729


In [13]:
result = {'Model': model_type, 'Training Accuracy': training_score, 'Training Time': training_time, "Test Accuracy": test_score,
          "Prediction Time": prediction_time}
df = pd.DataFrame(result)

#print("Total Process Time: {}".format(process_end -  process_start))
df


,Model,Training Accuracy,Training Time,Test Accuracy,Prediction Time
0,GaussianNB,0.587783,1.141622,0.5856,0.657928
1,RandomForest,0.99515,10.502508,0.8546,0.071329
2,GradientBoostingClassifier,0.9029,5638.623037,0.8669,0.621780
3,nnMLP,0.90255,265.021065,0.8688,0.101179
4,GaussianNB,0.587783,0.768270,0.5856,0.691740
5,RandomForest,0.9956,11.132927,0.8537,0.085664
6,GradientBoostingClassifier,0.9029,5901.939538,0.8668,0.393626
7,nnMLP,0.920067,223.916884,0.8809,0.099865
8,keras_adam,"[0.011163142539572974, 0.9968333333333333]",1177.883724,"[1.15368981669005, 0.8943]",0.341279
9,keras_sgd,"[0.038993409626434244, 0.9930833333333333]",946.155392,"[0.3552287856295705, 0.8973]",0.364729
